In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('insurance_pre.csv')

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype=int)

In [5]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
indep=dataset[['age', 'bmi', 'children','sex_male', 'smoker_yes']]
dep=dataset['charges']

In [7]:
#RF
from sklearn.ensemble import RandomForestRegressor
#SVR
from sklearn.svm import SVR
#DR
from sklearn.tree import DecisionTreeRegressor

In [9]:
from sklearn.model_selection import GridSearchCV
#RF
param_gridRF = {'criterion':['squared_error','absolute_error','friedman_mse','poisson'],'max_features': ['sqrt','log2',1.0], 'n_estimators':[10,100]}
#SVR
param_gridSVR = {'kernel':['sigmoid','linear'],'C':[10,100,1000,2000,3000],'gamma':['auto','scale']} #'rbf','poly',
#DR
param_gridDR = {'criterion':['squared_error','friedman_mse','absolute_error','poisson'],'max_features': ['sqrt','log2',1.0], 'splitter':['best','random']}

In [10]:
gridRF = GridSearchCV(RandomForestRegressor(), param_gridRF, refit = True, verbose = 3,n_jobs=-1)
gridRF.fit(indep, dep)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'absolute_error',
                                       'friedman_mse', 'poisson'],
                         'max_features': ['sqrt', 'log2', 1.0],
                         'n_estimators': [10, 100]},
             verbose=3)

In [11]:
#slowest of all 
gridSVR = GridSearchCV(SVR(), param_gridSVR, refit = True, verbose = 3,n_jobs=-1)
gridSVR.fit(indep, dep)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['sigmoid', 'linear']},
             verbose=3)

In [12]:
gridDR = GridSearchCV(DecisionTreeRegressor(), param_gridDR, refit = True, verbose = 3,n_jobs=-1)
gridDR.fit(indep, dep)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_features': ['sqrt', 'log2', 1.0],
                         'splitter': ['best', 'random']},
             verbose=3)

In [13]:
reRF=gridRF.cv_results_
print("The R_score value for best parameter {}:".format(gridRF.best_params_))

The R_score value for best parameter {'criterion': 'absolute_error', 'max_features': 'sqrt', 'n_estimators': 100}:


In [14]:
reSVR=gridSVR.cv_results_
print("The R_score value for best parameter {}:".format(gridSVR.best_params_))

The R_score value for best parameter {'C': 1000, 'gamma': 'auto', 'kernel': 'linear'}:


In [15]:
reDR=gridRF.cv_results_
print("The R_score value for best parameter {}:".format(gridDR.best_params_))

The R_score value for best parameter {'criterion': 'absolute_error', 'max_features': 1.0, 'splitter': 'random'}:


In [19]:
tableRF=pd.DataFrame.from_dict(reRF)
tableSVR=pd.DataFrame.from_dict(reSVR)
tableDR=pd.DataFrame.from_dict(reDR)

In [38]:
tableRF
# pick only the value that has ranking 1
df = pd.DataFrame(tableRF)
rowvalues = df[tableRF['rank_test_score']==1]
rowvalues

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,1.460664,0.026136,0.021821,0.007868,absolute_error,sqrt,100,"{'criterion': 'absolute_error', 'max_features'...",0.856873,0.781551,0.861082,0.831602,0.843258,0.834873,0.028619,1


In [39]:
# pick only the value that has ranking 1
df = pd.DataFrame(tableSVR)
rowvalues = df[tableSVR['rank_test_score']==1]
rowvalues

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
9,0.786610,0.109872,0.018467,0.004991,1000,auto,linear,"{'C': 1000, 'gamma': 'auto', 'kernel': 'linear'}",0.736398,0.675162,0.680002,0.699377,0.677952,0.693778,0.022955,1
11,0.777637,0.084451,0.017550,0.007654,1000,scale,linear,"{'C': 1000, 'gamma': 'scale', 'kernel': 'linear'}",0.736398,0.675162,0.680002,0.699377,0.677952,0.693778,0.022955,1


In [40]:
# pick only the value that has ranking 1
df = pd.DataFrame(tableDR)
rowvalues = df[tableDR['rank_test_score']==1]
rowvalues

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,1.460664,0.026136,0.021821,0.007868,absolute_error,sqrt,100,"{'criterion': 'absolute_error', 'max_features'...",0.856873,0.781551,0.861082,0.831602,0.843258,0.834873,0.028619,1


In [41]:
# pick only the value that has ranking 1
df = pd.DataFrame(tableRF)
rowvalues = df[tableRF['rank_test_score']==1]
rowvalues

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,1.460664,0.026136,0.021821,0.007868,absolute_error,sqrt,100,"{'criterion': 'absolute_error', 'max_features'...",0.856873,0.781551,0.861082,0.831602,0.843258,0.834873,0.028619,1


In [42]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:")) 
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 32
BMI: 33
Children: 1
Sex Male 0 or 1: 1
Smoker Yes 0 or 1: 1


In [47]:
print("Predictions")
Future_PredictionRF=gridRF.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
Future_PredictionSVR=gridSVR.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
Future_PredictionDR=gridRF.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Future_Prediction RF={}".format(Future_PredictionRF))
print("Future_Prediction SVR={}".format(Future_PredictionSVR))
print("Future_Prediction DR={}".format(Future_PredictionDR))
print("Highest predicted value for the same data set from different models",max(Future_PredictionRF,Future_PredictionSVR,Future_PredictionDR))

Predictions
Future_Prediction RF=[37305.16486735]
Future_Prediction SVR=[29555.98590468]
Future_Prediction DR=[37305.16486735]
Highest predicted value for the same data set from different models [37305.16486735]


C:\Anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
